### Setup:



In [1]:
!pip install -q sentence-transformers
!pip install -q wikipedia-api
!pip install -q numpy
!pip install -q scipy
!pip install -q flash_attn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 36.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


### Load the Embedding Model:

In [2]:
from sentence_transformers import SentenceTransformer
# This model supports two prompts: "s2p_query" and "s2s_query" for sentence-to-passage and sentence-to-sentence tasks, respectively.
# They are defined in `config_sentence_transformers.json`
# For the latest model refer: https://huggingface.co/spaces/mteb/leaderboard
query_prompt_name = "s2p_query"
model = SentenceTransformer("dunzhang/stella_en_1.5B_v5", trust_remote_code=True).cuda() # remove cuda() for using on cpu only.

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/174k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

...

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


1_Pooling/config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

2_Dense_1024/config.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

### Fetch Text Content from Wikipedia:

In [3]:
from wikipediaapi import Wikipedia
wiki = Wikipedia('RAGDemo/0.1', 'en')
doc = wiki.page('Masashi_Kishimoto').text
paragraphs = doc.split('\n\n') # chunking according to paragraphs

In [4]:
import textwrap
for i, paragraph in enumerate(paragraphs):
  wrapped_text = textwrap.fill(paragraph, width=100)
  print("-" * 100)
  print(wrapped_text)
  print("-" * 100)

----------------------------------------------------------------------------------------------------
Masashi Kishimoto (岸本 斉史, Kishimoto Masashi, born November 8, 1974) is a Japanese manga artist. His
manga series, Naruto, which was in serialization from 1999 to 2014, has sold over 250 million copies
worldwide in 46 countries as of May 2019. The series has been adapted into two anime and multiple
films, video games, and related media. Besides the Naruto manga, Kishimoto also personally
supervised the two canonical anime films, The Last: Naruto the Movie and Boruto: Naruto the Movie,
and has written several one-shot stories. In 2019, Kishimoto wrote Samurai 8: The Tale of Hachimaru
which ended in March 2020. From May 2016 through October 2020 he supervised the Boruto: Naruto Next
Generations manga written by Ukyō Kodachi and illustrated by Mikio Ikemoto. In November 2020 it was
announced that he had taken over as writer on the series, replacing Kodachi. A reader of manga from
a young ag

### Embed the Document:

In [5]:
embedded_docs = model.encode(paragraphs, normalize_embeddings=True) # working as an in memory Vector Database
shape = embedded_docs.shape

In [6]:
print("Number of chunks: " + str(shape[0]) + " Embedding Dimension: " + str(shape[1]))

Number of chunks: 22 Embedding Dimension: 1024


### Embed The Query:

In [7]:
query = "What is the most famous creation of Masashi Kishimoto"
embedded_query = model.encode(query, normalize_embeddings=True)
embedded_query.shape

(1024,)

### Find the Closest Chunks to the Query:

In [8]:
import numpy as np
similarities = np.dot(embedded_docs, embedded_query)

In [9]:
similarities.shape

(22,)

In [10]:
similarities

array([0.6711898 , 0.61047363, 0.5367813 , 0.55135703, 0.5092596 ,
       0.5010116 , 0.5403722 , 0.5726598 , 0.51858455, 0.4934883 ,
       0.5346582 , 0.4711318 , 0.5350033 , 0.5226188 , 0.51405895,
       0.51219463, 0.52122265, 0.55754685, 0.4297242 , 0.5504484 ,
       0.5685208 , 0.63115156], dtype=float32)

In [11]:
top_3_idx = np.argsort(similarities)[-3:][::-1].tolist() # argsort return s list of indexes according to their sorted version.

In [12]:
top_3_idx

[0, 21, 1]

In [25]:
CONTEXT = ""
for idx in top_3_idx:
    wrapped_text = textwrap.fill(paragraphs[idx], width=100)
    print("-" * 100)
    print(wrapped_text)
    print("-" * 100)
    CONTEXT += wrapped_text + "\n\n"


----------------------------------------------------------------------------------------------------
Masashi Kishimoto (岸本 斉史, Kishimoto Masashi, born November 8, 1974) is a Japanese manga artist. His
manga series, Naruto, which was in serialization from 1999 to 2014, has sold over 250 million copies
worldwide in 46 countries as of May 2019. The series has been adapted into two anime and multiple
films, video games, and related media. Besides the Naruto manga, Kishimoto also personally
supervised the two canonical anime films, The Last: Naruto the Movie and Boruto: Naruto the Movie,
and has written several one-shot stories. In 2019, Kishimoto wrote Samurai 8: The Tale of Hachimaru
which ended in March 2020. From May 2016 through October 2020 he supervised the Boruto: Naruto Next
Generations manga written by Ukyō Kodachi and illustrated by Mikio Ikemoto. In November 2020 it was
announced that he had taken over as writer on the series, replacing Kodachi. A reader of manga from
a young ag

In [26]:
prompt = f"""
use the following CONTEXT to answer the QUESTION at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

CONTEXT: {CONTEXT}
QUESTION: {query}

"""

In [27]:
!pip install -q -U google-generativeai

In [28]:
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

In [29]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [30]:
# Displays all the available models that are free to use, though they are subject to rate limits.
for m in genai.list_models():
    print(m.name, m.supported_generation_methods)

models/chat-bison-001 ['generateMessage', 'countMessageTokens']
models/text-bison-001 ['generateText', 'countTextTokens', 'createTunedTextModel']
models/embedding-gecko-001 ['embedText', 'countTextTokens']
models/gemini-1.0-pro-latest ['generateContent', 'countTokens']
models/gemini-1.0-pro ['generateContent', 'countTokens']
models/gemini-pro ['generateContent', 'countTokens']
models/gemini-1.0-pro-001 ['generateContent', 'countTokens', 'createTunedModel']
models/gemini-1.0-pro-vision-latest ['generateContent', 'countTokens']
models/gemini-pro-vision ['generateContent', 'countTokens']
models/gemini-1.5-pro-latest ['generateContent', 'countTokens']
models/gemini-1.5-pro-001 ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-pro ['generateContent', 'countTokens']
models/gemini-1.5-pro-exp-0801 ['generateContent', 'countTokens']
models/gemini-1.5-flash-latest ['generateContent', 'countTokens']
models/gemini-1.5-flash-001 ['generateContent', 'countTokens', 'createC

In [31]:
model = genai.GenerativeModel('gemini-1.5-flash')
# Calculates total number of tokens used for the api call
model.count_tokens(prompt)

total_tokens: 1044

In [32]:
response = model.generate_content(prompt)
print(response.text)

The most famous creation of Masashi Kishimoto is **Naruto**. 

